In [2]:
! pip install -r requirements.txt --quiet

# Connecting to a Remote MCP Server with Semantic Kernel

This notebook demonstrates how to connect to a remote MCP Server using **Semantic Kernel's** `MCPStreamableHttpPlugin`. The **Model Context Protocol (MCP)** enables scalable and modular tool integration across distributed systems. 

<br/>

> **Why Use Model Context Protocol (MCP)?**
>
>MCP allows agents to discover, invoke, and manage tools dynamically across remote servers.  
>It promotes modularity, scalability, and separation of concerns, making it easier to maintain and extend AI systems as they grow in complexity.

In [23]:
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent,ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from dotenv import load_dotenv
from os import environ
import asyncio

from semantic_kernel.connectors.mcp import MCPStreamableHttpPlugin

load_dotenv(override=True)

kernel = Kernel()

kernel.add_service(AzureChatCompletion(
    service_id="chat",
    deployment_name=environ["AZURE_OPENAI_MODEL"],
    base_url=environ["AZURE_OPENAI_ENDPOINT"],
    api_key=environ["AZURE_OPENAI_API_KEY"] ))



In [24]:
instructions = """You are AutoSales Analyst, an AI agent specialized in analyzing automotive sales data.

Your objectives:
- Dynamically choose and call the appropriate tool(s) to answer user questions about sales, customers, or products.
- Always include customer names alongside IDs in any output.
- Always display monetary amounts in $USD (e.g., $123.45).
- Use filters and aggregations as needed to generate insights from sales orders, products, and customers.
- Compute totals, revenue, discounts, and other metrics from nested order data.
- Return structured, concise results suitable for analysis or reporting.
- Avoid hardcoding analytics; rely on the MCP tools and their parameters.
- Clarify ambiguous queries before performing analysis.
- Treat the tools as the source of truth; do not expose raw database internals.

Behavioral guidance:
- For customer-related questions, resolve names and IDs before analyzing orders.
- For product-related questions, resolve categories or IDs before analyzing sales.
- Use run_query only for advanced analysis not supported by other tools.

Example user questions:
- "Which products generated the most revenue in the EU last quarter?"
- "Who are the top 5 customers by total sales this month?"
- "How much discount was given per product category in Q2?"
- "List all orders for customer 42 containing product 7."
"""

In [25]:

messages = [
    "Which products generated the most revenue?",
    "Drill down into customer details product with the highest revenue.",
    "Which customer had the highest sales for this product?",
]

thread: ChatHistoryAgentThread | None = None

async with MCPStreamableHttpPlugin(
    name="sales",
    url=f"{environ['MCP_SERVER_URL']}",
) as sales_plugin:

    agent = ChatCompletionAgent(
        kernel=kernel, 
        name="SalesAgent", 
        instructions=instructions,
        plugins=[sales_plugin, ]
    )


    for msg in messages:
        print("------------------------------------------------\n")
        print(msg)
        response = await agent.get_response(messages=msg, thread=thread)
        print(response)
        print("\n------------------------------------------------\n")

------------------------------------------------

Which products generated the most revenue?
Based on the latest order data, the products generating the most revenue are summarized below:

### Revenue by Product
1. **Brake Pad**: $11,113.50
2. **Oil Filter**: $4,223.85
3. **Spark Plug**: $3,487.68

The **Brake Pad** stands out as the highest revenue generator. Let me know if further analysis is needed!

------------------------------------------------

------------------------------------------------

Drill down into customer details product with the highest revenue.
After analyzing the sales data, the product generating the highest revenue is **Brake Pad**. Below are the customers associated with the data and their details:

### Customers:
1. **AutoZone**
   - **Region:** NA
   - **Industry:** Distributor
   - **Account Manager:** Carol Lee

2. **Bosch**
   - **Region:** EU
   - **Industry:** OEM
   - **Account Manager:** David Wong

3. **Ford**
   - **Region:** NA
   - **Industry:** 